In [56]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random

  Activating environment at `~/homework/optimal_control/planar_quadruped_landing/Project.toml`


In [57]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

reference_trajectory

## Reference Trajectory

## Problem Definition

In [58]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.2

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb/2                  # xb
xinit[2] = sqrt(l1^2+l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -3.0                   # yb_dot
xinit[12] = -3.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb/2           # xb
xterm[2] = sqrt(l1^2+l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([1.0;1.0;1.0; 1.0;1.0; 1.0;1.0; 1.0;1.0;1.0; 1.0;1.0; 1.0;1.0])
R = Diagonal(fill(1e-3,4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [59]:
nlp.cinds

4-element Vector{UnitRange{Int64}}:
 1:14
 15:28
 29:448
 449:479

In [60]:
# Initial guess
Random.seed!(1)
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
Uguess = zeros(size(Uref))
Z0 = packZ(nlp, Xguess, Uguess);
# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...


This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   265366
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      554
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      479
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

([-0.22027120154645383, 0.4417929873723346, 0.11476947752660983, -0.001044524463737564, -0.08390268543887641, -0.4688888661501666, 0.27950878238373106, -0.05223367574215085, -2.9770990445090217, -0.22670863488005308  …  0.06731453712002894, -0.5367348873534037, 0.25678184926050074, 0.2382933268800983, 0.07880250747201781, -0.007153885116035236, 0.0031319294723795134, 0.0405811817759249, -0.09579189021380435, 0.06961249481623331], Ipopt.Optimizer)

In [61]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
  0.029728798453546174
  0.0382395967790608
 -0.059763447672823114
 -0.001044524463737564
 -0.08390268543887641
  0.031111133849833383
  0.22950878238373107
 -0.05223367574215085
  0.02290095549097826
 -0.22670863488005308
  0.052996557616674614
  0.043142152642291265
  0.05837082875687786
  0.09632716050381906

In [62]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
  0.006988294035825182
  0.07338834687716467
  0.04634387170013981
 -0.03537417603088734
  0.06731453712002894
 -0.03673488735340369
  0.25678184926050074
  0.2382933268800983
  0.07880250747201781
 -0.007153885116035236
  0.0031319294723795134
  0.0405811817759249
 -0.09579189021380435
  0.06961249481623331

EXIT: Restoration failed!

EXIT: Converged to a point of local infeasibility. Problem may be infeasible.

In [63]:
display(Z_sol)
using DelimitedFiles
using CSV
writedlm( "data.csv",  Z_sol, ',')

554-element Vector{Float64}:
 -0.22027120154645383
  0.4417929873723346
  0.11476947752660983
 -0.001044524463737564
 -0.08390268543887641
 -0.4688888661501666
  0.27950878238373106
 -0.05223367574215085
 -2.9770990445090217
 -0.22670863488005308
  ⋮
 -0.5367348873534037
  0.25678184926050074
  0.2382933268800983
  0.07880250747201781
 -0.007153885116035236
  0.0031319294723795134
  0.0405811817759249
 -0.09579189021380435
  0.06961249481623331